In [16]:
#pip install keras
#pip install -U scikit-learn
#pip install pillow
#pip install matplotlib
#pip install jupyternotify
#unzip DR\ no\ aug
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [17]:
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

In [ ]:
"""
Notes for tmrw

u reduced batch size to 16 and u saw some improvements still plateuing
now ur pissed off coz accuracy is not increasing even though u put 0 data augmentation
tmrw I have to look into that"""

## Variables Being Used

In [43]:
#variables being used
batch_size=16
#val_split=0.2 #20 percent validation
image_size=512 
image_shape=(image_size,image_size)
#file_directory='/root/DR no aug'
#file_directory='/root/01DR_noDR'
train_file_directory='/root/DR-detection-using-OCT-A/train_val_split/Training'
validation_file_directory='/root/DR-detection-using-OCT-A/train_val_split/Validation'
alt_train_file_directory='/root/DR-detection-using-OCT-A/train_val_split/Training_no_weights'

In [44]:
!nvidia-smi

Sat Apr  1 20:56:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:61:00.0 Off |                  Off |
| 30%   30C    P2    66W / 450W |  23403MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Importing Datasets

In [61]:

from keras.applications.vgg16 import preprocess_input #for that vgg16
#import tensorflow as tf

#from keras.applications.Xception import preprocess_input

training_datagen = ImageDataGenerator( 
        preprocessing_function=preprocess_input, # use this only when using VGG16
        #rescale=1./255,
        #rotation_range=15,#can change this
        #shear_range=0.1,#can change this
        #zoom_range=0.1, #can change this
        #horizontal_flip=True,
        #fill_mode='constant',# constant,reflect,wrap,
        )

validation_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    #rescale=1./255    
        )

#Creating our generators, no need of changing values here
train_generator = training_datagen.flow_from_directory( #Creating our training generator 
        train_file_directory,  
        target_size=image_shape,
        batch_size=batch_size,
        #color_mode='grayscale',
        class_mode='categorical',
        shuffle = False,
        )

validation_generator = validation_datagen.flow_from_directory( #Creating our validation generator
        validation_file_directory, 
        target_size=image_shape, 
        batch_size=batch_size,
        #color_mode='grayscale',
        class_mode='categorical',
        shuffle = False,
)

Found 561 images belonging to 3 classes.
Found 50 images belonging to 3 classes.


## Displaying Images

In [62]:
# Enable/Disable to see 4 random images
"""fig, axs=plt.subplots(ncols=4, figsize=(20,20))
for i in range(4):
  axs[i].imshow(train_generator[i][0][0], cmap='gray', vmin=0, vmax=1)
#axs[0].plt.show()"""

"fig, axs=plt.subplots(ncols=4, figsize=(20,20))\nfor i in range(4):\n  axs[i].imshow(train_generator[i][0][0], cmap='gray', vmin=0, vmax=1)\n#axs[0].plt.show()"

## Creating weights

In [63]:
import numpy as np ## Calculating the class weights
from sklearn.utils.class_weight import compute_class_weight

class_weighing = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights=dict(zip(np.unique(train_generator.classes), class_weighing))

print(class_weights)

{0: 0.6051779935275081, 1: 0.9739583333333334, 2: 3.1166666666666667}


## Creating Model

In [64]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras.optimizers import SGD
import keras

In [65]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(image_size, image_size, 3),
    include_top=False)

#base_model.trainable = False
inputs = keras.Input(shape=(image_size, image_size, 3))

for layer in base_model.layers[:15]:
   layer.trainable = False
for layer in base_model.layers[15:]:
   layer.trainable = True

x = base_model(inputs)

x = keras.layers.GlobalAveragePooling2D()(x)
#x = keras.layers.Dropout(0.15)(x)  # Regularize with dropout
#x=keras.layers.Dense(100)(x)
#x=keras.layers.Dense(100)(x)
outputs = keras.layers.Dense(3)(x)
model = keras.Model(inputs, outputs)


model.summary(expand_nested=True)

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 512, 512, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 16, 16, 512)       14714688  
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_25 (InputLayer)     [(None, 512, 512, 3)]     0         |
|                                                               |
| block1_conv1 (Conv2D)     (None, 512, 512, 64)      1792      |
|                                                               |
| block1_conv2 (Conv2D)     (None, 512, 512, 64)      36928     |
|                                                               |
| block1_pool (MaxPooling2D)  (None, 256, 256, 64)    0         |
|                                                               |
| block2_conv1 (Conv2D)     (None, 256, 256, 128)     7385

### Compile

In [66]:
model.compile(optimizer=keras.optimizers.Adam(), #SGD or adam
              loss='categorical_crossentropy' ,
              metrics=['accuracy'])

### Generating Callbacks

In [67]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="/root/saved_models/{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint= ModelCheckpoint(filepath,monitor='val_accuracy',verbose=0,save_best_only=True,mode='max')
early_stop=EarlyStopping(monitor='val_accuracy',patience=50,verbose=0) #only change patience

callback_list=[checkpoint,early_stop]#add the callbacks used

### Model.Fit()

In [ ]:
%%notify
history=model.fit(
        train_generator,
        steps_per_epoch=len(train_generator.classes)//batch_size,
        verbose=1, 
        epochs=100,  #change this
        validation_data=validation_generator,
        validation_steps=len(validation_generator.classes)//batch_size,
        callbacks=callback_list,
        class_weight=class_weights)
print("Done bro")

Epoch 1/100


2023-04-01 21:01:21.975130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


35/35 [==============================] - ETA: 0s - loss: 5.8505 - accuracy: 0.3725

2023-04-01 21:01:29.263045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


35/35 [==============================] - 8s 222ms/step - loss: 5.8505 - accuracy: 0.3725 - val_loss: 2.6863 - val_accuracy: 0.4167
Epoch 2/100
35/35 [==============================] - 8s 214ms/step - loss: 7.1574 - accuracy: 0.3229 - val_loss: 13.4317 - val_accuracy: 0.4167
Epoch 3/100
35/35 [==============================] - 7s 212ms/step - loss: 10.7745 - accuracy: 0.3523 - val_loss: 13.4317 - val_accuracy: 0.4167
Epoch 4/100
35/35 [==============================] - 8s 213ms/step - loss: 11.0609 - accuracy: 0.3523 - val_loss: 13.4317 - val_accuracy: 0.4167
Epoch 5/100
35/35 [==============================] - 7s 212ms/step - loss: 10.6000 - accuracy: 0.3229 - val_loss: 13.4317 - val_accuracy: 0.4167
Epoch 6/100
35/35 [==============================] - 8s 213ms/step - loss: 10.6000 - accuracy: 0.3229 - val_loss: 13.4317 - val_accuracy: 0.4167
Epoch 7/100
35/35 [==============================] - 8s 214ms/step - loss: 10.6000 - accuracy: 0.3229 - val_loss: 13.4317 - val_accuracy: 0.4167


### Curves

In [ ]:
#loss curve
fig, axs=plt.subplots(ncols=2,figsize=(15, 5))#, figsize=(20,20))
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
axs[0].plot(epochs, loss, 'y', label='Training loss')
axs[0].plot(epochs, val_loss, 'r', label='Validation loss')
axs[0].set(title='Loss', xlabel='Epochs', ylabel='Epochs');

#accuracy curve
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
axs[1].plot(epochs, acc, 'y', label='Training acc')
axs[1].plot(epochs, val_acc, 'r', label='Validation acc')
axs[1].set(title='Accuracy', xlabel='Epochs', ylabel='Epochs');

In [ ]:
"""prediction_classes = np.array([])
true_classes =  np.array([])
i=0
for x, y in validation_generator:
    if i==len(validation_generator.classes):
        break    
    i=i+1
    prediction_classes = np.concatenate([prediction_classes,np.argmax(model.predict(x,verbose=0), axis = -1)])
    true_classes = np.concatenate([true_classes, np.argmax(y, axis=-1)])
print(classification_report(true_classes, prediction_classes))"""

In [ ]:
#model.save('to_continue.h5')
#continue_model=load_model('to_continue.h5')

## Evaluating Model

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np

#model = load_model('/root/44-0.82.hdf5')

In [ ]:
#new_model=load_model(".hdf5")

In [ ]:
data_eval=validation_generator
model_eval=model
results=model_eval.evaluate_generator(data_eval, steps=len(data_eval.classes)//batch_size)
print("Loss and accuracy are", results)
Y_pred = model_eval.predict_generator(data_eval, steps=len(data_eval.classes)//batch_size+1)
#print(train_generator.classes)
y_pred = np.argmax(Y_pred, axis=1)
#print(train_generator.classes)
#print(y_pred)
#print(X)
        
print('Confusion Matrix')
cm=confusion_matrix(data_eval.classes,y_pred)
target_names = ['NoDR', 'mildDR','seriousDR']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)
plt.show()

print('Classification Report')

print(classification_report(data_eval.classes,y_pred, target_names=target_names))